In [ ]:
%matplotlib inline
import network.network as Network
import network.mnist_loader as mnist_loader
import pickle
import matplotlib.pyplot as plt
import numpy as np

### Load Input Data

In [ ]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [ ]:
test_samples, test_labels = zip(*training_data)

### Load Trained Network

In [ ]:
with open('network/trained_network.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    net = u.load()

In [ ]:
sample_id = 0

sample = test_samples[sample_id]
sample_label = test_labels[sample_id]

In [ ]:
def generate_adversarial_sample(network, desired_label, steps, step_size):
    
    # generate random image
    x = np.random.normal(.5, .3, (784, 1))
    
    # modify image to desired adversarial sample
    for i in range(steps):
        derivative = net.input_derivative(x, desired_label)
        x -= step_size * derivative
    
    plt.imshow(x.reshape(28,28), cmap='Greys')
    return x
    

In [ ]:
adversarial_sample = generate_adversarial_sample(net, sample_label, 1000, 1)

### Select Detection Modules

### Run Experiment

In [ ]:
def predict(n):
    # Get the data from the test set
    x = list(test_data)[n][0]

    # Print the prediction of the network
    print('Network output: \n' + str(np.round(net.feedforward(x), 2)) + '\n')
    print('Network prediction: ' + str(np.argmax(net.feedforward(x))) + '\n')
    print('Actual image: ')
    
    # Draw the image
    plt.imshow(x.reshape((28,28)), cmap='Greys')

# Replace the argument with any number between 0 and 9999
predict(8342)